In [2]:
import ee
#import geetools
import numpy as np
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np

# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
# rgi outlines
asset_rgi01_Alaska = ee.FeatureCollection('projects/lzeller/assets/01_rgi60_Alaska')

# simple outline
asset_simpleoutline = ee.FeatureCollection('projects/lzeller/assets/AGVAsimplearea')  # eventually redo this to be areas within 5km of rgi outlines >0.5km

# subregion outlines
asset_subregions = ee.FeatureCollection('projects/lzeller/assets/Alaska_RGI_Subregions')

# define geometry to export
region = ee.Geometry.Polygon(  [[174.4484375,48.23114486338655],
                                [235.6203125,48.23114486338655],
                                [235.6203125,68.27298364423518],
                                [174.4484375,68.27298364423518],
                                [174.4484375,48.23114486338655]] )

In [4]:
# load the climate data, select variables and dates
climate_data = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
                    .filterDate('2018-01-01', '2023-01-01')
                    .select(['temperature_2m', 'total_precipitation_sum'])
               )

In [5]:
test = climate_data.select('temperature_2m').toBands()
# print(test.bandNames().getInfo())
print(test.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [0.1, 0, -180.05, 0, -0.1, 90.05]}


In [6]:
print(region.projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [7]:
# select temperature_2m, total_precipitation_sum and export separately
temp = climate_data.select('temperature_2m').toBands()
precip = climate_data.select('total_precipitation_sum').toBands()

# export the images to drive
task = ee.batch.Export.image.toDrive(
        image = temp, #regional_clipped_image,
        region = region, #asset_simpleoutline.geometry(), # region.bounds()
        folder = 'Climate',
        maxPixels = int(1e13),
        description = 'era5land_daily_temp_2018_2022',
        skipEmptyTiles = True
        )
task.start()

task = ee.batch.Export.image.toDrive(
        image = precip, #regional_clipped_image,
        region = region, #asset_simpleoutline.geometry(), # region.bounds()
        folder = 'Climate',
        maxPixels = int(1e13),
        description = 'era5land_daily_precip_2018_2022',
        skipEmptyTiles = True
        )
task.start()